In [5]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload  
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import os
import sys

import csv
import json
import time
import types, copy, pprint
import logging 

from datetime import datetime

if './src' not in sys.path:
    print(f"insert ./src")
    sys.path.insert(0, './src')
print(sys.path)

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from typing import Dict, List, Tuple
from scipy.sparse import csr_matrix
import shutil
import getpass
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torchinfo import summary
print(torch.__version__)

pd.options.display.width = 132
pp = pprint.PrettyPrinter(indent=4)
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=150, profile=None, sci_mode=None)

os.environ["WANDB_NOTEBOOK_NAME"] = "Adashare_Train.ipynb"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# print("1- '%x_%X'               : ",time.strftime('%x_%X'))
# print("2- '%X.%f'               : ",datetime.now().strftime('%X.%f'))
# print("3- '%X %x %Z'            : ",time.strftime('%X %x %Z'))
# print("4- '%D-%X.%f'            : ",datetime.now().strftime('%D-%X.%f'))
# print("5- '%Y-%m-%d %H:%M:%S.%f': ",datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f'))
# print("6- '%Y%m%d_%H%M%S'       : ",datetime.now().strftime('%Y%m%d_%H%M%S'))
# print("7- '%Y_%m_%d_%H:%M:%S'   : ",datetime.now().strftime('%Y_%m_%d_%H:%M:%S'))
print("7- '%Y_%m_%d_%H:%M:%S'   : ",datetime.now().strftime('%Y%m%d_%H%M%S'))

timestamp = datetime.now().strftime('%Y_%m_%d_%H:%M:%S')
print(' Timestamp: ','./tmp/sql_cohort'+'_'+timestamp+'.txt')

logLevel = os.environ.get('LOG_LEVEL', 'INFO').upper()
FORMAT = '%(asctime)s - %(levelname)s: - %(message)s'
logging.basicConfig(level="INFO", format= FORMAT)

['./src', '/home/kevin/miniforge3/envs/ptsnnl/lib/python311.zip', '/home/kevin/miniforge3/envs/ptsnnl/lib/python3.11', '/home/kevin/miniforge3/envs/ptsnnl/lib/python3.11/lib-dynload', '', '/home/kevin/miniforge3/envs/ptsnnl/lib/python3.11/site-packages']
2.2.1+cu118
7- '%Y_%m_%d_%H:%M:%S'   :  20240406_004122
 Timestamp:  ./tmp/sql_cohort_2024_04_06_00:41:22.txt


In [7]:
def _get_device():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = f"{device}:{torch.cuda.current_device()}"
    print("Running on:", device, torch.cuda.get_device_name(), torch.cuda.current_device())
    return device

current_device = _get_device()
print(f"Current device is : {current_device}")

Running on: cuda:0 Quadro GV100 0
Current device is : cuda:0


## Setup

In [8]:
# Soft Nearest Neighbor Loss
# Copyright (C) 2020  Abien Fred Agarap
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Affero General Public License as published
# by the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Affero General Public License for more details.
#
# You should have received a copy of the GNU Affero General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.
"""Sample module for using DNN classifier with SNNL"""
# import argparse
# import torch
# from pt_datasets import create_dataloader, load_dataset
# from pt_datasets import display_cellpainting_batch
# from snnl.models import DNN
from snnl.utils import export_results, get_hyperparameters, set_global_seed
from snnl.utils.metrics import accuracy
from snnl.utils import load_model, save_model, import_results, export_results, _save_checkpoint, _load_previous_state

__author__ = "Abien Fred Agarap"
__version__ = "1.0.0"

'Sample module for using DNN classifier with SNNL'

In [9]:
from dev_code import Model, DNN, Autoencoder, SNNLoss
from dev_code import CellpaintingDataset, InfiniteDataLoader, custom_collate_fn, parse_args

In [10]:
# if __name__ == "__main__":
input_args = f" --seed              1234 " \
             f" --model             snnl" \
             f" --configuration     examples/hyperparameters/dnn_cellpainting_unsupervised.json"
             # f" --configuration     examples/hyperparameters/dnn_cellpainting_classifier.json"
             # f" --configuration     examples/hyperparameters/dnn_mnist.json"
             # f" --model             baseline" \
args = parse_args(input_args.split())
args
set_global_seed(args.seed)

Namespace(seed=1234, model='snnl', configuration='examples/hyperparameters/dnn_cellpainting_unsupervised.json')

### main(args)

In [11]:
# DNN
(   dataset,
    batch_size,
    epochs,
    learning_rate,
    units,
    snnl_factor,
    temperature,
    cellpainting_args
 ) = get_hyperparameters(args.configuration)


hello common cellpainting hyperparameters
loading dnn hyperparameters
loading dnn cellpainting hyperparameters


In [12]:
# dataset = 'cellpainting'
# units = [[1471, 1024], [1024, 1024], [1024, 512], [512,32] ]
# epochs = 10
batch_size = 1
learning_rate = 0.001
compounds_per_batch = 100
temperatureLR = 1e-3
# temperature = 1/temperature

if dataset == "cellpainting": 
    cellpainting_args['batch_size']  = batch_size
    cellpainting_args['dataset_path'] = '/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1471_HashOrder.csv'
    # cellpainting_ds = dict()
    # cellpainting_args['train_start'] = 0
    cellpainting_args['train_end']     = 120_000
    cellpainting_args['compounds_per_batch']   = compounds_per_batch
    # cellpainting_args['sample_size'] = 3
    # cellpainting_args['test_start']  = 300000
    # cellpainting_args['test_end']    = cellpainting_ds['test_start'] + 3333  ## 346542
    # cellpainting_args['chunksize']   = None
    # cellpainting_args['conversions'] = None
    # cellpainting_args['iterator'] = True

In [13]:
file_sz = cellpainting_args['train_end'] - cellpainting_args['train_start']
smp_sz = cellpainting_args['sample_size']
buf_sz = cellpainting_args['compounds_per_batch']
batch_size = 1
bth_sz = batch_size

recs_per_batch = smp_sz * bth_sz * buf_sz
bth_per_epoch = file_sz // recs_per_batch
print(f" - Each batch contains {recs_per_batch/smp_sz} compounds, {smp_sz} samples per each compound : total {recs_per_batch} rows")
print(f" - Number of {recs_per_batch} row full size batches per epoch: {bth_per_epoch}")
print(f" - Rows covered by {bth_per_epoch} full size batches ({recs_per_batch} rows) per epoch:  {(file_sz // recs_per_batch) * recs_per_batch}")
print(f" - Last batch contains : {file_sz % recs_per_batch} rows")
 

 - Each batch contains 100.0 compounds, 3 samples per each compound : total 300 rows
 - Number of 300 row full size batches per epoch: 400
 - Rows covered by 400 full size batches (300 rows) per epoch:  120000
 - Last batch contains : 0 rows


In [14]:
# print(f"   code_dim          {code_dim}")
print(f"   dataset           {dataset}")
print(f"   batch_size        {batch_size}")
print(f"   epochs            {epochs}")
print(f"   learning_rate     {learning_rate}")
print(f"   units             {units}")
print(f"   snnl_factor       {snnl_factor}")
print(f"   temperature       {temperature}")
print(f"   temperatureLR:    {temperatureLR}")
print()
if dataset == "cellpainting": 
    pp.pprint(cellpainting_args)

   dataset           cellpainting
   batch_size        1
   epochs            10
   learning_rate     0.001
   units             [[1471, 1024], [1024, 1024], [1024, 512], [512, 128], [128, 128]]
   snnl_factor       10.0
   temperature       50.0
   temperatureLR:    0.001

{   'batch_size': 1,
    'chunksize': None,
    'compounds_per_batch': 100,
    'conversions': None,
    'dataset_path': '/home/kevin/WSL-shared/Cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1471_HashOrder.csv',
    'iterator': True,
    'sample_size': 3,
    'test_end': 310000,
    'test_start': 300000,
    'train_end': 120000,
    'train_start': 0}


## Define dataset and dataloaders

In [15]:
#### Load CellPainting Dataset
if dataset == 'cellpainting':
    print(f" load {dataset}")
    train_dataset = CellpaintingDataset(train = True,    **cellpainting_args)
    train_loader = InfiniteDataLoader(dataset=train_dataset, batch_size=batch_size,shuffle = False, num_workers = 0, collate_fn = custom_collate_fn)
    

 load cellpainting


In [16]:
    # train_dataset, test_dataset = load_dataset(name='cellpainting', cellpainting_args = cellpainting_args)
    # test_loader = create_dataloader(dataset=test_dataset, batch_size=batch_size, shuffle = False, num_workers = 0 , collate_fn = custom_collate_fn)
    # train_dataloader = DataLoader(dataset = train_dataset, batch_size=10, num_workers = 0, collate_fn=custom_collate, worker_init_fn = worker_init_fn, shuffle= False)
    # test_dataloader  = DataLoader(dataset = test_dataset , batch_size=12, num_workers = 0, collate_fn=custom_collate, worker_init_fn = worker_init_fn, shuffle= False)

# for idx, batch in enumerate(train_loader):
#     print(f"{idx:4d}, {batch[0].shape}, {batch[1].shape},{batch[2].shape},{batch[3].shape},{batch[4].shape}, {batch[3][:3]}, {batch[3][-3:]}")
#     train_batch_id +=1
    # display_cellpainting_batch(train_batch_id, batch)
    # if idx >= 2:
        # break
    # break

### define DNN model

In [17]:
if args.model.lower() == "baseline":
    print(f" baseline model")
    model = DNN(units=units, 
                learning_rate=learning_rate)
elif args.model.lower() == "snnl":
    print(f" snnl model")
    model = DNN(
        units=units,
        learning_rate=learning_rate,
        use_snnl=True,
        factor=snnl_factor,
        temperature=temperature,
        use_annealing= False,
        unsupervised = True,
        sample_size = cellpainting_args['sample_size']
    )
else:
    raise ValueError("Choose between [baseline] and [snnl] only.")

model.device
model.optimizer
# model.optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
# model.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(model.optimizer, mode='min', factor=0.5, patience=30, 
#                                                              threshold=0.00001, threshold_mode='rel', cooldown=5, 
#                                                              min_lr=0, eps=1e-08)
# model.scheduler.get_last_lr()

 snnl model
 Building Base Model from NOTEBOOK
    Model_init()_    -- Crtierion  :       CrossEntropyLoss()
    Model_init()_    -- temperature :      50.0
    Model_init()_    -- temperature LR:    0.001
    Model_init()_    -- mode:              classifier
    Model_init()_    -- unsupervised :     True
    Model_init()_    -- use_snnl :         True
 Build SNNLoss dfrom NOTEBOOK
 Building SNNLoss from NOTEBOOK
    SNNLoss _init()_    -- mode: classifier was found in SNNLoss._supported_modes --   is unsupervised: False
    SNNLoss _init()_    -- primary_criterion: CrossEntropyLoss()
    SNNLoss _init()_    -- unsupervised :     True
    SNNLoss _init()_    -- use_annealing :    False
    SNNLoss _init()_    -- sample_size :      3
 Building DNN from NOTEBOOK
    DNN _init()_    -- mode:              classifier
    DNN _init()_    -- unsupervised :     True
    DNN _init()_    -- use_snnl :         True
    DNN _init()_    -- temperature :      Parameter containing:
tensor([50.], dev

device(type='cuda', index=0)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

#### Display model summary

In [ ]:
col_names = [ "input_size", "output_size", "num_params", "params_percent", "mult_adds", "trainable"]
             # "kernel_size"
summary_input_size = (batch_size * cellpainting_args['sample_size'], 1471)
summary(model, input_size=summary_input_size, col_names = col_names)

In [ ]:
activations = {}
for index, layer in enumerate(model.layers):
    # print(f" layer {index}    {layer}")  
    if index == 0:
        print(f"a- layer {index} activations[{index}] : Relu(layer[{index}])                   Input: {layer.in_features}   out: {layer.out_features}")   
    elif index == len(model.layers) - 1:
        print(f"c- layer {index} activations[{index}] : Layer[{index}](activations[{index-1}])         Input: {layer.in_features}   out: {layer.out_features}")   
    else:
        print(f"b- layer {index} activations[{index}] : Relu(Layer[{index}](activations[{index-1}]))   Input: {layer.in_features}   out: {layer.out_features}")   


### `model.fit(data_loader=train_loader, epochs=epochs)`

In [18]:
if model.use_snnl:
    model.train_snn_loss = []
    model.train_xent_loss = []
    model.train_temp_hist = []
    model.train_temp_grad_hist = []

In [19]:
show_every = 1
starting_epoch, epoch , epochs = 0,0,10

In [20]:
starting_epoch, epoch , epochs

(0, 0, 10)

In [21]:
print(f" run epochs {starting_epoch+1} to {epochs} ")

 run epochs 1 to 10 


In [22]:
# model.temperature , model.snnl_criterion.temperature 
# model.temperature = 40.0
# model.snnl_criterion.temperature = model.temperature 
# model.temperature
# model.snnl_criterion.temperature 
# model.optimizer.param_groups[0]["lr"]
# from torch import linalg as LA
# LA.vector_norm(model.temperature.grad)
# LA.vector_norm(model.snnl_criterion.temperature.grad)

In [37]:
print(f"{datetime.now().strftime('%Y%m%d_%H%M%S')}  epoch {starting_epoch+1:4d} of {epochs:4d}")

for epoch in range(starting_epoch,epochs):
    
    fit_epoch_loss, fit_epoch_accuracy = model.epoch_train( train_loader, epoch, factor = 10, verbose = False)

    model.train_loss.append(fit_epoch_loss[0])
    model.train_snn_loss.append(fit_epoch_loss[1])
    model.train_xent_loss.append(fit_epoch_loss[2])
    model.train_accuracy.append(fit_epoch_accuracy)
    model.train_temp_hist.append(model.temperature.item())
    model.train_temp_grad_hist.append(model.temperature.grad.item())

    if (epoch + 1) % show_every == 0:
        print(f"{datetime.now().strftime('%Y%m%d_%H%M%S')}  epoch {epoch + 1:4d} of {epochs:4d}"
              f"  mean loss = {model.train_loss[-1]:.6f} |  mean acc = {model.train_accuracy[-1]:.6f} |"
              f"  xent loss = {model.train_xent_loss[-1]:.6f} |  snn loss = {model.train_snn_loss[-1]:.6f} |"
              f"  temp = {model.temperature.item():.6f}  |  temp grad = {model.temperature.grad.item():.6f} ")

    # model.scheduler.step(model.train_loss[-1])

20240406_065913  epoch  101 of  150
20240406_070012  epoch  101 of  150  mean loss = 49.278454 |  mean acc = 0.000000 |  xent loss = 0.000000 |  snn loss = 49.278454 |  temp = 5.235223  |  temp grad = 0.169523 
20240406_070111  epoch  102 of  150  mean loss = 49.176104 |  mean acc = 0.000000 |  xent loss = 0.000000 |  snn loss = 49.176104 |  temp = 4.604747  |  temp grad = 0.208398 
20240406_070209  epoch  103 of  150  mean loss = 49.037272 |  mean acc = 0.000000 |  xent loss = 0.000000 |  snn loss = 49.037272 |  temp = 3.927939  |  temp grad = 0.284722 
20240406_070308  epoch  104 of  150  mean loss = 48.841936 |  mean acc = 0.000000 |  xent loss = 0.000000 |  snn loss = 48.841936 |  temp = 3.182779  |  temp grad = 0.407278 
20240406_070406  epoch  105 of  150  mean loss = 48.504949 |  mean acc = 0.000000 |  xent loss = 0.000000 |  snn loss = 48.504949 |  temp = 2.300226  |  temp grad = 0.709154 
20240406_070505  epoch  106 of  150  mean loss = 47.742974 |  mean acc = 0.000000 |  xent

In [27]:
starting_epoch, epochs, epoch

(10, 25, 24)

In [32]:
# epochs = 80 
epochs = epochs + 50
starting_epoch = epoch + 1

print(f" run epochs {starting_epoch+1} to {epochs}  -- {epoch}")

 run epochs 101 to 150  -- 99


#### SNNL Outputs

In [ ]:
model.snnl_criterion.layers_snnl
# model.snnl_criterion.snnl_loss

In [ ]:
model.snnl_criterion.pairwise_distance_matrix[:9,:9]

In [ ]:
model.snnl_criterion.pick_probability[:9,:9]

In [ ]:
model.snnl_criterion.masked_pick_probability[:9,:9]
9.4345e-04+  6.9088e-04+  2.6218e-05

In [ ]:
(model.snnl_criterion.summed_masked_pick_probability)

In [ ]:
# np.log(1.8488e-05)

In [ ]:
torch.log(model.snnl_criterion.summed_masked_pick_probability) 

In [ ]:
# from sklearn.preprocessing import .normalize(
# a = torch.Tensor([[2],[1],[2],[3],[2],[9]]) 
a.shape 
# b = torch.Tensor([[3],[4],[2],[4],[5],[5]])
a = torch.Tensor([2,1,2,3,2,9])
b = torch.Tensor([3,4,2,4,5,5])
product = torch.matmul(a, b)
product
normalized_a = torch.nn.functional.normalize(a, dim=0, p=2)
normalized_b = torch.nn.functional.normalize(b, dim=0, p=2)
normalized_a
normalized_b
normalized_b = torch.conj(normalized_b).T
normalized_b
product = torch.matmul(normalized_a, normalized_b)
product
distance_matrix = torch.sub(torch.tensor(1.0), product)
distance_matrix

In [ ]:
a = torch.tensor([[0.9041,  0.0196], [-0.3108, -2.4423], [-0.4821,  1.059]])
a.shape
torch.cdist(a, a, p=2)

In [ ]:
starting_epoch, epochs, epoch

## Plot losses

In [ ]:
model_attributes = model.__dict__
for key, value in model_attributes.items():
    if isinstance(value, List) or "test_accuracy" in key:
        print(f"{key:25s} {type(value)}  {len(value):7d}  {value[:5]}")
# model_attributes['temperature_gradients'] 
temp_gradients = np.array(model_attributes['temperature_gradients'])

In [ ]:
model.train_snn_loss

In [ ]:
for st in range(0,len(temp_gradients), 1000):
    end = st + 1000
    print(f" {st:5d} - {end:5d}  min: {temp_gradients[st:end].min():9e}   max: {temp_gradients[st:end].max():9e}    avg: {temp_gradients[st:end].mean():9e}   std: {temp_gradients[st:end].std():9e}")

In [ ]:
_ = plt.plot(model.train_snn_loss);
_ = plt.title(f'train_snn_loss - {epochs} epochs');
plt.show()

In [ ]:
# plt.plot(model.train_snn_loss)
# plt.title(f'train_snn_loss - {epochs} epochs')
# plt.show()

In [ ]:
_ = plt.plot(model.train_temp_hist);
_ = plt.title(f'train_temperature_hist - {epochs} epochs');
plt.show()

In [ ]:
_ = plt.plot(model.temperature_gradients)
_ = plt.title(f'train_temperature_gradients')
plt.show()

In [ ]:
_ = plt.plot(model.train_temp_grad_hist);
_ = plt.title(f"train_temperature_gradient_hist - {epochs} epochs");
plt.show()

#### Plot Temperature Gradients

In [ ]:
plt.plot(model.train_temp_grad_hist[:38])
plt.title(f'train_temperature_gradient_hist')
plt.show()
plt.plot(model.train_temp_grad_hist[38:42])
plt.title(f'train_temperature_gradient_hist')
plt.show()
plt.plot(model.train_temp_grad_hist[42:])
plt.title(f'train_temperature_gradient_hist')
plt.show()

In [ ]:
# plt.plot(model.train_loss)

plt.plot(model.train_temp_hist)
plt.title(f'train_temperature_hist')
plt.show()
plt.plot(model.train_temp_grad_hist[:38])
plt.title(f'train_temperature_gradient_hist')
plt.show()
plt.plot(model.train_temp_grad_hist[38:42])
plt.title(f'train_temperature_gradient_hist')
plt.show()
plt.plot(model.train_temp_grad_hist[42:])
plt.title(f'train_temperature_gradient_hist')
plt.show()


In [ ]:
plt.plot(model.temperature_gradients)
plt.show()
plt.plot(model.temperature_gradients[:23600])
plt.show()
plt.plot(model.temperature_gradients[:43600])
plt.show()
plt.plot(model.temperature_gradients[43600:48600])
plt.show()

plt.plot(model.temperature_gradients[250000:])

In [ ]:
# plt.show()
# plt.plot(model.temperature_gradients[43600:44600])
# plt.show()
# plt.plot(model.temperature_gradients[44600:45600])
# plt.show()
# plt.plot(model.temperature_gradients[45600:46600])
# plt.show()
# plt.plot(model.temperature_gradients[46600:47600])
# plt.show()
# plt.plot(model.temperature_gradients[47600:47700])
# plt.show()
# plt.plot(model.temperature_gradients[47700:47800])
# plt.show()
plt.plot(model.temperature_gradients[47800:47900])
plt.show()
# plt.plot(model.temperature_gradients[47900:48100])
# plt.show()
# plt.plot(model.temperature_gradients[48100:48300])
# plt.show()
# plt.plot(model.temperature_gradients[48300:48600])
# plt.show()
# plt.plot(model.temperature_gradients[48600:49600])
# plt.show()

## Save model results and model

In [35]:
# import datetime.timezone
# datetime.timetz()
timestamp = datetime.now().strftime('%Y_%m_%d_%H%M%S')
filename = f"DNN_{args.model.lower()}_lrn_temp_epoch_{epochs}_cpb_{compounds_per_batch}_clipgrad_factor_{10}_{timestamp}"
print(f"saved to {filename}")

saved to DNN_snnl_lrn_temp_epoch_150_cpb_100_clipgrad_factor_10_2024_04_06_065852


In [36]:
export_results(model=model, filename=filename)

save_model(model, filename)

_save_checkpoint(epochs, model, filename, update_latest=False, update_best=False)

[INFO] Model Results exported to results/DNN_snnl_lrn_temp_epoch_150_cpb_100_clipgrad_factor_10_2024_04_06_065852.json.
[INFO] Model exported to examples/export/DNN_snnl_lrn_temp_epoch_150_cpb_100_clipgrad_factor_10_2024_04_06_065852.pt.
[INFO] Model exported to ckpts/DNN_snnl_lrn_temp_epoch_150_cpb_100_clipgrad_factor_10_2024_04_06_065852.pt.


## Load model

In [ ]:
from snnl.utils import load_model, _load_previous_state, import_results

In [ ]:
fn = "DNN_snnl_withstatictemp_epoch400_2024_04_01_073006.pt"
fn = "DNN_snnl_withlearning_temp_epoch250_2024_04_02_173536_model_epoch_250.pth"
model_clone = load_model(fn)

In [ ]:
model_clone

In [ ]:
# fn = "DNN_snnl_withstatictemp_epoch400_2024_04_01_073006.pt"
fn = "DNN_snnl_withlearning_temp_epoch250_2024_04_02_173536_model_epoch_250.pth"
model, next_epoch = _load_previous_state(model, fn)

In [ ]:
next_epoch
model_clone.

#### Load model results

In [ ]:
# fn = "DNN_snnl_withstatictemp_epoch400_2024_04_01_073006.pt"
fn = "DNN_snnl_withlearning_temp_epoch250_2024_04_02_173536.json"

results = import_results(fn)

In [ ]:
results.keys()
for k in results.keys():
    print(len(results[k]))
    model.__dict__[k] = results[k]

## Predict

In [ ]:
if dataset == 'cellpainting':
    print(f" load {dataset}")
    test_dataset = CellpaintingDataset(test = True, **cellpainting_args)
    test_loader = InfiniteDataLoader(dataset=test_dataset, batch_size=batch_size, shuffle = False, num_workers = 0, collate_fn = train_dataset.custom_collate_fn)
    

In [ ]:
for idx, batch in enumerate(test_loader):
    print(idx, batch[0].shape, batch[1].shape,batch[2].shape,batch[3].shape,batch[4].shape)
    # test_features = batch[0].reshape(-1, 784)
    # print(batch[1])
    # display_cellpainting_batch(idx, batch)
    # train_batch_id +=1
    # if train_batch_id ==3 :
        # break
    # if idx >= 24:
        # break
    break

In [ ]:
# test_features = test_dataset.dataa.reshape(-1, 784) / 255.0
test_features = batch[0] 

In [ ]:
model.eval()
model = model.cpu()

In [ ]:
predictions = model.predict(test_features)
model.test_accuracy = accuracy(y_true=test_dataset.targets, y_pred=predictions)

In [ ]:
print(f"accuracy: {model.test_accuracy}%")

In [ ]:
print(f"accuracy: {model.test_accuracy}%")

In [ ]:
print(f"accuracy: {model.test_accuracy}%")

In [ ]:
filename = f"DNN-{args.model.lower()}-{args.seed}.json"
export_results(model=model, filename=filename)

## `epoch_train(self, data_loader, epoch)` - STEP BY STEP

    def epoch_train(self, data_loader: torch.utils.data.DataLoader, epoch: int = None) -> Tuple:

In [ ]:
model.snnl_criterion = SNNLoss(
        mode=model.mode,
        criterion=model.primary_criterion,
        factor=model.factor,
        temperature=model.temperature,
        use_annealing=model.use_annealing,
        use_sum=model.use_sum,
        # unsupervised=model.unsupervised,
        unsupervised=True,
        code_units=model.code_units,
        sample_size = model.sample_size,
        stability_epsilon=model.stability_epsilon,
)
model.unsupervised

In [ ]:
# if model.use_snnl:
# if model.name == "DNN" or model.name == "CNN":
epoch_loss = 0
epoch_ttl_loss = 0
epoch_primary_loss = 0
epoch_snn_loss = 0
epoch_accuracy = 0

# model.snnl_criterion.factor 
# model.snnl_criterion.factor = 2
# model.snnl_criterion.factor 

#### Loop through data

In [ ]:
for batch_count, (batch_features, batch_labels, _, _, _) in enumerate(train_loader):
    if batch_count  == 71:
        break
    # ,_,_,batch_features, batch_labels = next(iter(data_loader))
    # batch_features, batch_labels = next(iter(data_loader))

    # if model.name in ["Autoencoder", "DNN"]:
        # batch_features = batch_features.view(batch_features.shape[0], -1)

In [ ]:
batch_count

In [ ]:
    
batch_features = batch_features.to(model.device)
batch_labels = batch_labels.to(model.device)
    
model.optimizer.zero_grad()
    
outputs = model.forward(features=batch_features)
    # break

In [ ]:
# outputs

In [ ]:
    if model.use_snnl:
        train_loss, primary_loss, snn_loss = model.snnl_criterion(
            model=model,
            outputs=outputs,
            features=batch_features,
            labels=batch_labels,
            epoch=epoch,
        )
        epoch_loss += train_loss.item()
        epoch_snn_loss += snn_loss.item()
        epoch_primary_loss += primary_loss.item()
    else:
        # if not model.use_snnl:
        print(f"Model not using SNNL")
        train_loss = model.criterion(outputs, batch_labels if model.name == "DNN" or model.name == "CNN"  else batch_features,)
        epoch_loss += train_loss.item()

In [ ]:
 
    if model.name == "DNN" or model.name == "CNN":
        train_accuracy = (outputs.argmax(1) == batch_labels).sum().item() / len(batch_labels)
        epoch_accuracy += train_accuracy
    else:
        train_accuracy =  0
        epoch_accuracy += train_accuracy
    
    train_loss.backward()
    model.optimizer.step()
    
    if model.use_snnl and model.temperature is not None:
        model.optimize_temperature()
    
    # print(outputs.shape)
    # print(batch_labels.shape)
    print(f" batch:{batch_count} - train : loss    {train_loss:10.6f}   primary loss: {primary_loss:10.6f}    SNN loss: {snn_loss*model.snnl_criterion.factor:10.6f}   (loss: {snn_loss:10.6f} * factor: {model.snnl_criterion.factor})  "
          f"  temp: {model.temperature.data} snnl_temp: {model.snnl_criterion.temperature.data}")    
    # print(f" epoch : loss    {epoch_loss:10.6f}   primary_loss: {epoch_primary_loss:10.6f}    SNN loss: {epoch_snn_loss:10.6f}   factor: {model.factor}")
    # print(f" batch_accuracy         {train_accuracy}   epoch_accuracy         {epoch_accuracy}")
    break
#### End of dataloader loop

In [ ]:
print(f" loop ended - batch_count: {batch_count}")

In [ ]:
primary_loss
primary_loss.shape, primary_loss.size(), primary_loss.data, primary_loss.ndim

In [ ]:
pp = torch.zeros(0,requires_grad = True, dtype=torch.float32)
p1 = torch.tensor(0,requires_grad = True, dtype=torch.float32, device = model.device)
pp.shape, pp.size(), pp.data, pp.ndim
p1.shape, p1.size(), p1.data, p1.ndim

In [ ]:
len_data_loader = batch_count

In [ ]:
len_data_loader
epoch_loss, epoch_loss/len_data_loader
epoch_snn_loss * model.snnl_criterion.factor, (epoch_snn_loss * model.snnl_criterion.factor)/len_data_loader, epoch_snn_loss
epoch_primary_loss, epoch_primary_loss/len_data_loader
epoch_accuracy / len_data_loader

## SNNL Loss

In [ ]:
# input1 = torch.randn(100, 128)
# input2 = torch.randn(100, 128)
# cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
# tmp1 = cos(input1, input2)
# tmp1.shape
# # tmp1

# value.shape

# tmp1 = torch.cdist(value, value, p=2)
# cosim = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
# tmp1 = cosim(value, value.T)
# tmp1.shape
# tmp1
# tmp1.min(), tmp1.argmin()

In [ ]:
snnl =SNNLoss(
                mode=model.mode,
                criterion=model.primary_criterion,
                factor=model.factor,
                temperature=model.temperature,
                use_annealing=model.use_annealing,
                use_sum=model.use_sum,
                code_units=model.code_units,
                sample_size = model.sample_size,
                stability_epsilon=model.stability_epsilon,
                unsupervised=model.unsupervised,
            )

In [ ]:
snnl.temperature = 8.762743

In [ ]:
model.temperature
snnl.temperature

In [ ]:
activations = snnl.compute_activations(model=model, features=batch_features)

In [ ]:
key = 1
value = activations[1]
value.shape

In [ ]:
def pairwise_cosine_distance(features: torch.Tensor) -> torch.Tensor:
    """
    Returns the pairwise cosine distance between two copies
    of the features matrix.

    Parameter
    ---------
    features: torch.Tensor
        The input features.

    Returns
    -------
    distance_matrix: torch.Tensor
        The pairwise cosine distance matrix.

    Example
    -------
    >>> import torch
    >>> from snnl import SNNLoss
    >>> _ = torch.manual_seed(42)
    >>> a = torch.rand((4, 2))
    >>> snnl = SNNLoss(temperature=1.0)
    >>> snnl.pairwise_cosine_distance(a)
    tensor([[1.1921e-07, 7.4125e-02, 1.8179e-02, 1.0152e-01],
            [7.4125e-02, 1.1921e-07, 1.9241e-02, 2.2473e-03],
            [1.8179e-02, 1.9241e-02, 1.1921e-07, 3.4526e-02],
            [1.0152e-01, 2.2473e-03, 3.4526e-02, 0.0000e+00]])
    """
    a, b = features.clone(), features.clone()
    normalized_a = torch.nn.functional.normalize(a, dim=1, p=2)
    normalized_b = torch.nn.functional.normalize(b, dim=1, p=2)
    # normalized_b = torch.conj(normalized_b).T
    product = torch.matmul(normalized_a, normalized_b.T)
    distance_matrix = torch.sub(torch.tensor(1.0), product)
    return distance_matrix

    def normalize_distance_matrix(
        self,
        features: torch.Tensor,
        distance_matrix: torch.Tensor,
        device: torch.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    ) -> torch.Tensor:

        pairwise_distance_matrix = torch.exp( -(distance_matrix * self.temperature)) - torch.eye(features.shape[0]).to(device)
        
        return pairwise_distance_matrix

In [ ]:
def pairwise_cos_distance(A, B):
    
    """Pairwise cosine distance between two matrices.
    :param A: a matrix.
    :param B: a matrix.
    :returns: A tensor for the pairwise cosine between A and B.
    """
    normalized_A = torch.nn.functional.normalize(A, dim=1)
    normalized_B = torch.nn.functional.normalize(B, dim=1)
    prod = torch.matmul(normalized_A, normalized_B.transpose(-2, -1).conj())
    return 1 - prod

### V1 Code

In [ ]:
print('\n','-'*80)
print('pairwise_cos_distance')
print('-'*80)

tmp1 =  pairwise_cosine_distance(value)
tmp1.shape
tmp1
tmp1.min(), tmp1.argmin()


print('\n','-'*80)
print('pairwise_distance_matrix')
print('-'*80)

tmp2 = torch.exp(-(tmp1 / snnl.temperature)) - torch.eye(batch_features.shape[0]).to(model.device) 
tmp2.shape
tmp2
tmp2.min(), tmp2.argmin()



In [ ]:
snnl.distance_matrix = snnl.pairwise_cosine_distance(features = value)
snnl.distance_matrix.shape
snnl.distance_matrix
snnl.distance_matrix.min(), snnl.distance_matrix.argmin()

In [ ]:
snnl.pairwise_distance_matrix = snnl.normalize_distance_matrix(features = value, distance_matrix = snnl.distance_matrix, device = 'cuda:0')
snnl.pairwise_distance_matrix
snnl.pairwise_distance_matrix.shape   
snnl.pairwise_distance_matrix.min(), snnl.pairwise_distance_matrix.argmin()   
# pairwise_distance_matrix
# tmp1 = torch.exp(-(snnl.distance_matrix / snnl.temperature)) 
# tmp1.shape
# tmp1.max(), tmp1.argmax()
# tmp1.min(), tmp1.argmin()

In [ ]:
snnl.pick_probability = snnl.compute_sampling_probability(snnl.pairwise_distance_matrix)
snnl.pick_probability.shape   
snnl.pick_probability
# snnl.pick_probability.sum(0), snnl.pick_probability.sum(1) 
snnl.pick_probability.min(), snnl.pick_probability.argmin()   

In [ ]:
snnl.summed_masked_pick_probability = snnl.mask_sampling_probability(labels = batch_labels, sampling_probability = snnl.pick_probability, device = 'cuda:0')
snnl.summed_masked_pick_probability.shape
snnl.summed_masked_pick_probability.sum(0)
snnl.summed_masked_pick_probability.min(), snnl.summed_masked_pick_probability.argmin()

In [ ]:
snnl_loss = torch.mean( -torch.log(snnl.stability_epsilon + snnl.summed_masked_pick_probability))
snnl_loss

### V2 Code

In [4]:
from snnl.losses import SNNLCrossEntropy

In [ ]:
snnlV2 = SNNLCrossEntropy(model,
                         temperature=100.,
                         layer_names=[0,1,2,3,4],
                         factor=-10.,
                         optimize_temperature=True,
                         cos_distance=True)

In [ ]:
# snnlV2.to('cuda:0')

In [ ]:
tmp1 = snnlV2.SNNL(x = value.to('cpu') ,
              y = snnl.sample_labels.to('cpu') ) 
# snnl.sample_labels
tmp1
# torch.arange(batch_labels.shape[0]).to('cuda:0') // snnl.sample_size

In [ ]:
temp = snnl.temperature
temp
sample_labels = snnl.sample_labels.to('cpu') 
# sample_labels

In [ ]:
print('\n','-'*80)
print('pairwose_cos_distance')
print('-'*80)
tmp2 = snnlV2.pairwise_cos_distance(value, value)
tmp2.shape
tmp2
tmp2.min(), tmp2.argmin()

print('\n','-'*80)
print('torch.exp(-(distance_matrix / temp))')
print('-'*80)
tmp3 = torch.exp(-(tmp2 / temp))
tmp3.shape
tmp3
tmp3.min(), tmp3.argmin()

print('\n','-'*80)
print(' torch.exp(-(distance_matrix / temp)) - torch.eye(value.shape[0] ')
print('-'*80)
tmp4 = tmp3 - torch.eye(value.shape[0]).to('cuda')
tmp4.shape
tmp4
tmp4.min(), tmp4.argmin()

print('\n','-'*80)
print(' PickProbability --- tmp4 / (SNNLCrossEntropy.STABILITY_EPS + tmp4.sum(axis=1).unsqueeze(1))')
print('-'*80) 
tmp5 = tmp4 / (SNNLCrossEntropy.STABILITY_EPS + tmp4.sum(axis=1).unsqueeze(1))
tmp5.shape
tmp5[:10,:10]
tmp5
tmp5.min(), tmp5.argmin()

print('\n','-'*80)
print(' MaskedPickProbability ')
print('-'*80) 
tmp6 = tmp5.to('cpu') * snnlV2.same_label_mask(sample_labels, sample_labels )
tmp5.shape
tmp6[:10,:10]
tmp6
tmp6.min(), tmp6.argmin()

print('\n','-'*80)
print(' SummedMaskedPickProbability ')
print('-'*80) 
tmp7 = tmp6.sum(axis=1)
tmp7.shape
tmp7
tmp7.min(), tmp7.argmin()


print('\n','-'*80)
print(' SummedMaskedPickProbability ')
print('-'*80) 
tmp8 =  -torch.log(SNNLCrossEntropy.STABILITY_EPS + tmp7).mean()
tmp8.shape
tmp8
tmp8.min(), tmp8.argmin()


# summed_masked_pick_prob = SNNLCrossEntropy.masked_pick_probability(x, y, temp, cos_distance).sum(axis=1)
# snnlV2.same_label_mask(sample_labels, sample_labels)

 #### Print values used in snnl loss

In [ ]:
# outputs.shape
# outputs[0].data
# outputs.argmax(1)
batch_labels

# batch_labels[1]
# train_accuracy

In [ ]:
# torch.set_printoptions(linewidth=180)

# model.snnl_criterion.distance_matrix[:10,:10]

# model.snnl_criterion.distance_matrix[:10,:10]

# model.snnl_criterion.pairwise_distance_matrix[:10,:10]

# model.snnl_criterion.pick_probability[:10,:10]

# model.snnl_criterion.masking_matrix[:10,:10]

# model.snnl_criterion.masked_pick_probability[:10,:10]

# model.snnl_criterion.summed_masked_pick_probability[:10]

# tmp1 = torch.eq(model.snnl_criterion.sample_labels, model.snnl_criterion.sample_labels.unsqueeze(1)).float()
# tmp1.shape
# tmp1[:12,:12]

In [ ]:
# (outputs.argmax(1) == batch_labels).sum().item()
# len(batch_labels)
# len(data_loader)

# activations = dict()
# layers = model.layers
# for index, layer in enumerate(layers):
#     if index == 0:
#         activations[index] = layer(batch_features)
#     else:
#         activations[index] = layer(activations[index - 1])
# for i in activations:
#     print(activations[i].shape)

In [ ]:
# len_data_loader = batch_count -1
epoch_loss /= len_data_loader

if model.name in ["DNN", "CNN"]:
    epoch_accuracy /= len_data_loader ## len(data_loader)

if model.use_snnl:
    epoch_snn_loss = epoch_snn_loss * model.snnl_criterion.factor / len_data_loader ## len(data_loader)
    epoch_primary_loss /=  len_data_loader  ##  len(data_loader)
    
    if model.name == "DNN" or model.name == "CNN":
        print(f" epoch_loss: {epoch_loss},  epoch_snn_loss: {epoch_snn_loss}, epoch_primaey_loss: {epoch_primary_loss}, accuracy: {epoch_accuracy} ")
        a,b = (epoch_loss, epoch_snn_loss, epoch_primary_loss), epoch_accuracy
    else:
        print(f" epoch_loss: {epoch_loss},  epoch_snn_loss: {epoch_snn_loss}, epoch_primary_loss: {epoch_primary_loss} ")
        a =  (epoch_loss, epoch_snn_loss, epoch_primary_loss)
else:
    if model.name == "DNN" or model.name == "CNN":
        print(f" epoch_loss: {epoch_loss}, accuracy: {epoch_accuracy} ")
        a , b = epoch_loss, epoch_accuracy
    else:
        print(f" epoch_loss: {epoch_loss},")
        a =  epoch_loss